In [37]:
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.preprocessing import image

import os
import numpy as np

import networks as nw

In [38]:
# Instantiate the model
unet = nw.UNet(input_size=(256, 256, 1), num_classes=1)

# Summary of the model architecture
unet.build((None, 256, 256, 1))  # Input shape
unet.summary()

d:\lucab\Documents\Universita\Corsi\5o_anno\Heidelberg\Computer Vision (MVSpec)\Final Project\CV-skin-lesions\.venv\lib\site-packages\keras\src\layers\layer.py:393: UserWarning: `build()` was called on layer 'u_net_9', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "u_net_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_81 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_82 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_83 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_84 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_85 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_86 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_87 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_88 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_89 (Sequential)      │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_149 (Conv2D)             │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [39]:
def load_image(image_path):
    """Load an image, decode it, and resize to 256x256"""
    img = tf.io.read_file(image_path)               # Read image
    img = tf.image.decode_png(img, channels=3)       # Decode as PNG (assuming PNG images)
    img = tf.image.resize(img, (256, 256))           # Resize to target size
    img = img / 255.0                               # Normalize to range [0, 1]
    return img

def load_mask(mask_path):
    """Load a mask, decode it, and resize to 256x256"""
    mask = tf.io.read_file(mask_path)               # Read mask image
    mask = tf.image.decode_png(mask, channels=1)     # Decode as PNG (assuming masks are single-channel)
    mask = tf.image.resize(mask, (256, 256))         # Resize to target size
    mask = mask / 255.0                             # Normalize to range [0, 1]
    return mask

def load_data(image_folder, mask_folder):
    # Get image and mask file paths
    image_paths = [os.path.join(image_folder, fname) for fname in os.listdir(image_folder)]
    mask_paths = [os.path.join(mask_folder, fname) for fname in os.listdir(mask_folder)]
    
    # Shuffle the paths (optional)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))

    # Load images and masks
    dataset = dataset.map(lambda x, y: (load_image(x), load_mask(y)))
    
    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=1000)  # Shuffle with buffer_size (adjust as needed)
    dataset = dataset.batch(batch_size=16)  # Batch size for training (adjust as needed)
    
    return dataset


In [40]:
# Set the directory paths
image_folder = 'normalized_images'
mask_folder = 'normalized_masks'

# Load the dataset
train_dataset = load_data(image_folder, mask_folder)

# Check the dataset shapes (optional)
for images, masks in train_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Mask batch shape:", masks.shape)

# Compile the model
unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
unet.fit(train_dataset, epochs=10)  # Adjust the number of epochs as needed

Image batch shape: (16, 256, 256, 3)
Mask batch shape: (16, 256, 256, 1)
Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 256, 256, 1), output.shape=(None, 128, 128, 1)